### Transfer the pdf to text and dataframe

In [163]:
import pdftotext
import pandas as pd
import re
import numpy as np

In [80]:
# Load your PDF
with open("/Users/timliu/Documents/on the desktop/MSc BA/論文/ARP/European (Re)Insurers/HNR1 GY/20150310_Hannover_Rueck_SE-_Earnings_Call_2015-3-10_FS000000002198853065.pdf", "rb") as f:
    pdf = pdftotext.PDF(f)

# Save all text to a txt file.
with open('/Users/timliu/Desktop/output/test.txt', 'w') as f:
    f.write("\n\n".join(pdf))

# open the text file
with open('/Users/timliu/Desktop/output/test.txt') as f:
    contents = f.readlines()

# contents

# ToDo:
# write a function to processsing all the file

After extracting from the text, will do some preprocessing to the text.

In [81]:
# ### Cleaning all the unwanted rows in the transcript
# df = pd.DataFrame(contents)

# # remove the unnessary string
# df[0] = df[0].str.replace('\n','')
# df[0] = df[0].str.replace('Bloomberg Transcript','')
# df[0] = df[0].str.replace('\x0c\n','')
# df[0] = df[0].str.replace('FINAL','')
# df[0] = df[0].str.replace('A - ','')
# df[0] = df[0].str.replace('Q - ','')

# # using re to remove the unnessary string
# def drop_unnessary(x):
#     page = re.findall(r'Page \d+ of \d+', x) # 'page ... of ... '
#     BIO = re.findall(r'{BIO', x) # '{BIO 18731996 <GO>}'
#     Company_Name = re.findall(r'Company N ame:', x) # 'Company N ame: H annover Rueck SE'
#     Company_Ticker = re.findall(r'Company Ticker:', x) # 'Company Ticker: H N R1 GR Equity'
#     Date = re.findall(r'Date:', x) # Date: 2015-03-10
#     if page == [] and BIO == [] and Company_Name == [] and Company_Ticker == [] and Date == []:
#         return True
#     else:
#         return False

# true_false = df[0].apply(lambda x: drop_unnessary(x))
# df = df[true_false]

# # drop the final page declaration
# df = df[df[0] != 'This transcript may not be 100 percent accurate and may contain misspellings and other']
# df = df[df[0] != 'inaccuracies. This transcript is provided "as is", without express or implied warranties of']
# df = df[df[0] != 'any kind. Bloomberg retains all rights to this transcript and provides it solely for your']
# df = df[df[0] != 'personal, non-commercial use. Bloomberg, its suppliers and third-party agents shall']
# df = df[df[0] != 'have no liability for errors in this transcript or for lost profits, losses, or direct, indirect,']
# df = df[df[0] != 'incidental, consequential, special or punitive damages in connection with the']
# df = df[df[0] != 'furnishing, performance or use of such transcript. Neither the information nor any']
# df = df[df[0] != 'opinion expressed in this transcript constitutes a solicitation of the purchase or sale of']
# df = df[df[0] != 'securities or commodities. Any opinion expressed in the transcript does not necessarily']
# # df = df[df[0] != 'reflect the views of Bloomberg LP. ¬© COPYRIGHT 2022, BLOOMBERG LP. All rights']  
# df = df[df[0] != 'reserved. Any reproduction, redistribution or retransmission is expressly prohibited.']
# # ¬© could not be identified, would apply re
# def drop_Bloomberg_mark(x):
#     Bloomberg_mark = re.findall(r'reflect the views of Bloomberg LP', x) # 'reflect the views of Bloomberg LP. ¬© COPYRIGHT 2022, BLOOMBERG LP. All rights'
#     if Bloomberg_mark == []:
#         return True
#     else:
#         return False

# true_false = df[0].apply(lambda x: drop_Bloomberg_mark(x))
# df = df[true_false]


# # drop the empthy row
# df = df[df[0] != '']
# df = df[df[0] != '']


In [82]:
def cleaning_text(contents):
    ### Cleaning all the unwanted rows in the transcript
    df = pd.DataFrame(contents)

    # remove the unnessary string
    df[0] = df[0].str.replace('\n','')
    df[0] = df[0].str.replace('Bloomberg Transcript','')
    df[0] = df[0].str.replace('\x0c\n','')
    df[0] = df[0].str.replace('FINAL','')
    df[0] = df[0].str.replace('A - ','')
    df[0] = df[0].str.replace('Q - ','')

    # using re to remove the unnessary string
    def drop_unnessary(x):
        page = re.findall(r'Page \d+ of \d+', x) # 'page ... of ... '
        BIO = re.findall(r'{BIO', x) # '{BIO 18731996 <GO>}'
        Company_Name = re.findall(r'Company N ame:', x) # 'Company N ame: H annover Rueck SE'
        Company_Ticker = re.findall(r'Company Ticker:', x) # 'Company Ticker: H N R1 GR Equity'
        Date = re.findall(r'Date:', x) # Date: 2015-03-10
        if page == [] and BIO == [] and Company_Name == [] and Company_Ticker == [] and Date == []:
            return True
        else:
            return False

    true_false = df[0].apply(lambda x: drop_unnessary(x))
    df = df[true_false]

    # drop the final page declaration
    df = df[df[0] != 'This transcript may not be 100 percent accurate and may contain misspellings and other']
    df = df[df[0] != 'inaccuracies. This transcript is provided "as is", without express or implied warranties of']
    df = df[df[0] != 'any kind. Bloomberg retains all rights to this transcript and provides it solely for your']
    df = df[df[0] != 'personal, non-commercial use. Bloomberg, its suppliers and third-party agents shall']
    df = df[df[0] != 'have no liability for errors in this transcript or for lost profits, losses, or direct, indirect,']
    df = df[df[0] != 'incidental, consequential, special or punitive damages in connection with the']
    df = df[df[0] != 'furnishing, performance or use of such transcript. Neither the information nor any']
    df = df[df[0] != 'opinion expressed in this transcript constitutes a solicitation of the purchase or sale of']
    df = df[df[0] != 'securities or commodities. Any opinion expressed in the transcript does not necessarily']
    # df = df[df[0] != 'reflect the views of Bloomberg LP. ¬© COPYRIGHT 2022, BLOOMBERG LP. All rights']  
    df = df[df[0] != 'reserved. Any reproduction, redistribution or retransmission is expressly prohibited.']
    # ¬© could not be identified, would apply re
    def drop_Bloomberg_mark(x):
        Bloomberg_mark = re.findall(r'reflect the views of Bloomberg LP', x) # 'reflect the views of Bloomberg LP. ¬© COPYRIGHT 2022, BLOOMBERG LP. All rights'
        if Bloomberg_mark == []:
            return True
        else:
            return False

    true_false = df[0].apply(lambda x: drop_Bloomberg_mark(x))
    df = df[true_false]

    # drop the empthy row
    df = df[df[0] != '']
    df = df[df[0] != '']

    return df


In [83]:
df = cleaning_text(contents)
df

,0
4,Q4 2014 Earnings Call
5,Company Participants
8,Roland Vogel
9,Ulrich Wallin
10,Unidentified Participant
...,...
1511,"For all of those others here in the room, we w..."
1512,Hannover Re for a snack and a refreshment outs...
1513,attending. Thank you for coming.
1515,Ulrich Wallin


### Make a list of the participants in the meeting

In [84]:
def participants_list(df):
    # reset the index to make sure the index is continuous for better processing
    df = df.reset_index(drop=True)

    #  'Company Participants' index
    df.loc[df[0] == 'Company Participants']
    Participant_start_index = df.index[df[0] == 'Company Participants'].tolist()
    #  'Other Participants' index
    df.loc[df[0] == 'Other Participants']
    Participant_middle_index = df.index[df[0] == 'Other Participants'].tolist()
    #  'MANAGEMENT DISCUSSION SECTION' index, is the beginning of the management discussion, would stop before this row
    df.loc[df[0] == 'MANAGEMENT DISCUSSION SECTION']
    Participant_end_index = df.index[df[0] == 'MANAGEMENT DISCUSSION SECTION'].tolist()

    print(Participant_start_index, Participant_middle_index, Participant_end_index)

    # make the list of company_paticipants and other_participants
    company_paticipants = df.loc[Participant_start_index[0]+1:Participant_middle_index[0]-1]
    company_paticipants.drop(company_paticipants.index[company_paticipants[0] == ''].tolist(), inplace=True)

    other_paticipants = df.loc[Participant_middle_index[0]+1:Participant_end_index[0]-1]
    other_paticipants.drop(other_paticipants.index[other_paticipants[0] == ''].tolist(), inplace=True)

    print("==========================")
    print("the company paticipants is: ", company_paticipants)
    print("==========================")
    print("the other paticipants is: ", other_paticipants)

    # after extract the paticipants, we can drop those information to make the transcript more clear
    df = df.drop(range(df.index[df[0] == 'Company Participants'].tolist()[0],df.index[df[0] == 'MANAGEMENT DISCUSSION SECTION'].tolist()[0]+1))
    # Q4 2014 Earnings Call
    df.drop(df.index[df[0] == 'Q4 2014 Earnings Call'].tolist(), inplace=True)

    # reset the index again to make sure the index is continuous for better processing
    df = df.reset_index(drop=True)
    # save to csv
    df.to_csv('/Users/timliu/Desktop/output/df.csv')
    return df, company_paticipants, other_paticipants

In [95]:
df,company_paticipants,other_paticipants = participants_list(df)


[1] [6] [18]
the company paticipants is:                            0
2              Roland Vogel
3             Ulrich Wallin
4  Unidentified Participant
5    Unverified Participant
the other paticipants is:                       0
7    Andrew J. Ritchie
8            Ben Cohen
9      Frank Kopfinger
10  Janet Van den Berg
11      Kamran Hossain
12        Michael Haid
13   Olivia S. Brindle
14      Peter Casanova
15        Rötger Franz
16      Thomas Fossard
17       Vikram Gandhi


/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


### Spilt the text into different df by different participants

In [96]:
# list of both company_paticipants and other_participants
# change the company_paticipants to list 
company_paticipants_list = company_paticipants[0].tolist()
other_paticipants_list = other_paticipants[0].tolist()
# merge the list
both_participants_list = company_paticipants_list + other_paticipants_list

In [97]:
# # create a dict to store all the data
# participants_dict = {}
# for name in both_participants_list:
#     participants_dict[name] = pd.DataFrame()
# participants_dict

# for i in range(len(both_participants_list)):
#     participants_1_start = df[df[0] == both_participants_list[i]].index
#     collect_index = []
#     for i in range(len(participants_1_start)):
#         collect_index.append(both_participants_row_index.index(participants_1_start[i]))
#         # #  add 1 into the collect_index
#     collect_index_end = [x+1 for x in collect_index]
#     # get the value of both_participants_row_index by collect_index_end
#     participants_1_end = [both_participants_row_index[x] for x in collect_index_end]
#     participants_1_end

In [98]:
both_participants_list

['Roland Vogel',
 'Ulrich Wallin',
 'Unidentified Participant',
 'Unverified Participant',
 'Andrew J. Ritchie',
 'Ben Cohen',
 'Frank Kopfinger',
 'Janet Van den Berg',
 'Kamran Hossain',
 'Michael Haid',
 'Olivia S. Brindle',
 'Peter Casanova',
 'Rötger Franz',
 'Thomas Fossard',
 'Vikram Gandhi']

In [99]:
# list of both company_paticipants and other_participants
# change the company_paticipants to list 
company_paticipants_list = company_paticipants[0].tolist()
other_paticipants_list = other_paticipants[0].tolist()
# merge the list
both_participants_list = company_paticipants_list + other_paticipants_list

# identify all the rows in df with both_participants in it
both_participants_row_index = df[df[0].isin(both_participants_list)].index.tolist()
both_participants_row_index

# identify all the rows in df with both_participants_list[0] in it
participants_1_start = df[df[0] == both_participants_list[0]].index
participants_1_start


collect_index = []
for i in range(len(participants_1_start)):
    collect_index.append(both_participants_row_index.index(participants_1_start[i]))

collect_index
#  add 1 into the collect_index
collect_index_end = [x+1 for x in collect_index]
collect_index, collect_index_end

# get the value of both_participants_row_index by collect_index_end
participants_1_end = [both_participants_row_index[x] for x in collect_index_end]
participants_1_end

# participants_1_start to list
participants_1_start = participants_1_start.tolist()
print(both_participants_list[0])
print(participants_1_start, participants_1_end)
print(len(participants_1_start), len(participants_1_end))

Roland Vogel
[86, 584, 659, 785, 803, 845, 902, 924, 930, 934] [329, 604, 672, 792, 805, 856, 914, 928, 932, 936]
10 10


In [100]:
collect_index_end

[3, 23, 28, 40, 43, 51, 59, 62, 64, 66]

In [101]:
for i in range(1,len(participants_1_start)):
    # participants_1 = df.loc[participants_1_start[0]+1:participants_1_end[0]-1]
    participants_1 = df.loc[participants_1_start[0]+1:participants_1_end[0]-1]
    participants_merge = df.loc[participants_1_start[i]+1:participants_1_end[i]-1]
    participants_1 = pd.concat([participants_1, participants_merge])

# concat all the rows into singl row
participants_1['value'] = 1
participants_1 = participants_1.groupby('value')[0].apply(''.join).reset_index()

# drop the "value" column
participants_1.drop(participants_1.columns[0], axis=1, inplace=True)

# rename the column
participants_1.rename(columns={0: both_participants_list[0]}, inplace=True) #################################

# save to csv to check
participants_1.to_csv('/Users/timliu/Desktop/output/Andrew_J_Ritchie.csv')

# the company paticipants is:                            0
# 2              Roland Vogel
# 3             Ulrich Wallin
# 4  Unidentified Participant
# 5    Unverified Participant
# ==========================
# the other paticipants is:                       0
# 7    Andrew J. Ritchie
# 8            Ben Cohen
# 9      Frank Kopfinger
# 10  Janet Van den Berg
# 11      Kamran Hossain
# 12        Michael Haid
# 13   Olivia S. Brindle
# 14      Peter Casanova
# 15        Rötger Franz
# 16      Thomas Fossard
# 17       Vikram Gandhi

participants_1


,Roland Vogel
0,"Yes, thank you, Uli. The good thing is that I ..."


### testified the time and date extracted from the text

In [128]:
df_testing = df.iloc[100:600]
df_testing.rename(columns={0: 'name'}, inplace=True)
df_testing

,name
100,"results, even with a little bit of a more depr..."
101,positive.
102,"On the right hand side, the assets under own m..."
103,tell you with the development of the U.S. doll...
104,"government bonds, this today is approaching €3..."
...,...
595,"million have not been used. We, I think, addre..."
596,three quarters that some of that falls down a ...
597,"other areas. So, my guidance would not be that..."
598,redundancies. But what we see is that the – wh...


In [151]:
df_testing

,name
100,"results, even with a little bit of a more depr..."
101,positive.
102,"On the right hand side, the assets under own m..."
103,tell you with the development of the U.S. doll...
104,"government bonds, this today is approaching €3..."
...,...
595,"million have not been used. We, I think, addre..."
596,three quarters that some of that falls down a ...
597,"other areas. So, my guidance would not be that..."
598,redundancies. But what we see is that the – wh...


In [155]:
# find all the date and time in the transcript using re
def find_date_time(x):
    # find the month like 'March'
    # list of month
    month = [
        # month
        'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December',
        'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'
        ]
    # find the month
    month_list = re.findall(r'\b' + '|'.join(month) + '\b', x)
    # write back to the original df
    return month_list

    # # find the day like '1st'
    # # list of day
    # day = [
    #     '1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th',
    #     '9th', '10th', '11th', '12th', '13th', '14th', '15th',
    #     '16th', '17th', '18th', '19th', '20th', '21st', '22nd',
    #     '23rd', '24th', '25th', '26th', '27th', '28th', '29th',
    #     '30th', '31st'
    #     ]
    # # find the day
    # day_list = re.findall(r'\b(' + '|'.join(day) + ')\b', x)
    # # print the day_list
    # print(day_list)


# apply the function to the df_testing apply lambda
df_testing['date_time'] = df_testing['name'].apply(lambda x: find_date_time(x))


# # list of month
# month_list = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
# # list of short month
# short_month_list = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
# # find all the month_list or short_month_list inside the participants_1
# for i in range(len(month_list)):
#     participants_1[month_list[i]] = participants_1[0].str.contains(month_list[i])
#     participants_1[short_month_list[i]] = participants_1[0].str.contains(short_month_list[i])

# participants_1





/var/folders/sq/sxhsy98j6f57m406rd5lz1_r0000gn/T/ipykernel_89010/1066517891.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_testing['date_time'] = df_testing['name'].apply(lambda x: find_date_time(x))


In [162]:
np.unique(df_testing['date_time'])


array([list([]), list(['April']), list(['February']), list(['June']),
       list(['Mar']), list(['October']), list(['September'])],
      dtype=object)

In [135]:
month = [
    'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December',
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'
    ]


month_list = re.findall(r'\b(' + '|'.join(month) + ')\b', df_testing.iloc[19,0])


In [139]:
month_list

[]

In [144]:
for i in range(len(df2)):
    list = []
    list.append(find_date_time(df2.iloc[i,0]))
    

AttributeError: 'str' object has no attribute 'name'